In [33]:
# UOC - Visualització de dades - PRA2 
# Preparació de dades: importa CSV i agrupa
# @author: José V. Grimalt
# 2022-01-15

# import packages
import re
import collections
import pandas as pd
from datetime import datetime, timedelta


In [34]:
# noms dels fitxers
file_municipis = "municipios_pob_2022-01-01.csv"  # dades de població dels municipis

# càrrega de dades de municipis
municipis = pd.read_csv(file_municipis, index_col=0).reset_index()

# llista de fitxers CSV mensuals
file_input_csv = [                                
    f"csv/matriculacion_{year}{month:02d}.csv" 
        for year in range(2018, 2023) for month in range(1, 13) 
]

# camps de classifiació:
# fields_group = ['MOTOR', 'COMBUSTIBLE', 'TECNOLOGÍA', 'REG_EMISIONES']
fields_group = ['MOTOR', 'COMBUSTIBLE', 'TECNOLOGÍA']
fields_num = ['CO2', 'CANTIDAD', '%']

# fitxers de sortida:
file_group_tecno = "matriculacions_tecno_2018-2022.csv"
file_group_mensual = "matriculacions_2018-2022_mensual.csv"
file_group_co2 = "matriculacions_co2_2018-2022.csv"
file_group_total = "matriculacions_2018-2022_total.csv"
file_group_prov = "matriculacions_ev_provincia_2018-2022.csv"
file_dataset = "matricula_mensuals_españa_2018-2022.csv"

rows = collections.defaultdict(lambda: 0)


In [35]:
dict_comunitat = {
    "Alacant": "Comunitat Valenciana",
    "Albacete": "Castella la Manxa",
    "Almería": "Andalusia",
    "Araba": "País Basc",
    "Asturias": "Principat d'Astúries",
    "Badajoz": "Extremadura",
    "Balears (Illes)": "Illes Balears",
    "Barcelona": "Catalunya",
    "Bizkaia": "País Basc",
    "Burgos": "Castella i Lleó",
    "Cantabria": "Cantàbria",
    "Castelló": "Comunitat Valenciana",
    "Ceuta": "Ceuta",
    "Ciudad Real": "Castella la Manxa",
    "Coruña (A)": "Galícia",
    "Cuenca": "Castella la Manxa",
    "Cáceres": "Extremadura",
    "Cádiz": "Andalusia",
    "Córdoba": "Andalusia",
    "Gipuzkoa": "País Basc",
    "Girona": "Catalunya",
    "Granada": "Andalusia",
    "Guadalajara": "Castella la Manxa",
    "Huelva": "Andalusia",
    "Huesca": "Aragó",
    "Jaén": "Andalusia",
    "León": "Castella i Lleó",
    "Lleida": "Catalunya",
    "Lugo": "Galícia",
    "Madrid": "Comunitat de Madrid",
    "Melilla": "Melilla",
    "Murcia": "Regió de Murcia",
    "Málaga": "Andalusia",
    "Ourense": "Galícia",
    "Palencia": "Castella i Lleó",
    "Palmas (Las)": "Illes Canaries",
    "Pontevedra": "Galícia",
    "Rioja (La)": "La Rioja",
    "Salamanca": "Castella i Lleó",
    "Santa Cruz de Tenerife": "Illes Canaries",
    "Segovia": "Castella i Lleó",
    "Sevilla": "Andalusia",
    "Soria": "Castella i Lleó",
    "Tarragona": "Catalunya",
    "Teruel": "Aragó",
    "Toledo": "Castella la Manxa",
    "Valladolid": "Castella i Lleó",
    "València": "Comunitat Valenciana",
    "Zamora": "Castella i Lleó",
    "Zaragoza": "Aragó",
    "Ávila": "Castella i Lleó",
}


In [36]:
df = pd.DataFrame()

# Bucle principal per als fitxers mensuals

# for file_txt, file_csv in zip(file_input_txt, file_output_csv):
for file_csv in file_input_csv[:]:
    # print(file_csv)
    dades = pd.read_csv(file_csv)
    rows["TOTAL"] += len(dades)
    # begin: correcciones
    # dades = dades.rename(columns={"TECNOLOGIA": "TECNOLOGÍA"})
    # dades.to_csv(file_csv, sep=",", index=False)
    # end: correciones 
    df = pd.concat([df, dades], axis=0, ignore_index=True)

# df.info()


In [37]:
display(df.groupby(["VEHÍCULO", "MOTOR", ])[["CANTIDAD"]].sum())
rows["TOTAL2"] += len(df)
# filtrem dades per tipus de vehícle: "TURISMO"
# df = df[df["VEHÍCULO"]=="TURISMO"] # 51857
df = df[df["VEHÍCULO"].isin(["TURISMO","FURG./TODOTERR."])] # 77336
df = df.drop(columns=["VEHÍCULO"])
rows["TIPO"] += len(df)


CANTIDAD
VEHÍCULO          MOTOR               
AUTOBUS           Combustión     14040
                  Eléctrico        563
CAMION            Combustión    310961
                  Eléctrico       2090
CAMION ARTICULADO Combustión      9927
                  Eléctrico       1134
FURG./TODOTERR.   Combustión    528274
                  Eléctrico      29319
TURISMO           Combustión   5396170
                  Eléctrico     203814

In [38]:
print(len(df))
display(df.groupby(["MOTOR", "COMBUSTIBLE", "TECNOLOGÍA",])[["CANTIDAD"]].sum())


121064


CANTIDAD
MOTOR      COMBUSTIBLE    TECNOLOGÍA          
Combustión Combustión G/D Combustión      4470
           Diesel         Combustión   2214425
                          HEV           116628
           GLP            GLP            84743
                          HEV               58
           GNC            GNC            18348
           GNL            GNL               68
                          HEV                1
           Gasolina       Combustión   2808121
                          HEV           677394
           Otros          Combustión       132
                          HEV               56
Eléctrico  Diesel         PHEV            5982
           Eléctrico      BEV           101539
           GLP            PHEV               5
           GNC            PHEV               1
           Gasolina       PHEV          124327
                          REEV            1182
           Hidrógeno      Hidrógeno          9
                          PHEV              83
           Otros          BEV                1
                          PHEV               4

In [39]:
# df.groupby(["MOTOR", "TECNOLOGÍA", "REG_EMISIONES"])[["CO2"]].mean()

In [40]:
# calculem la mitjana d'emissions CO2 per vehicle:
df["CO2_MEDIA"] = df["CO2"] / df["CANTIDAD"]

df_t = df.groupby(["REG_EMISIONES", "MOTOR", "TECNOLOGÍA"])[["CO2","CANTIDAD"]].sum()
df_t["CO2_MEDIA"] = df_t["CO2"] / df_t["CANTIDAD"]
# resum de dades
print(
    "\nFilas totales: ", len(df),
    "\nFilas con valor 0 para CO2_MEDIA con REG_EMISIONES == 'SÍ': ",
    len(df[(df["CO2_MEDIA"]==0) & (df["REG_EMISIONES"]=="SÍ")]),
    "\nFilas con valor 0 para CO2_MEDIA con REG_EMISIONES == 'NO': ",
    len(df[(df["CO2_MEDIA"]==0) & (df["REG_EMISIONES"]=="NO")]),
    "\nVehículos totales: \n", 
    df_t,
)



Filas totales:  121064 
Filas con valor 0 para CO2_MEDIA con REG_EMISIONES == 'SÍ':  0 
Filas con valor 0 para CO2_MEDIA con REG_EMISIONES == 'NO':  23216 
Vehículos totales: 
                                              CO2  CANTIDAD   CO2_MEDIA
REG_EMISIONES MOTOR      TECNOLOGÍA                                   
NO            Combustión Combustión          0.0     28722    0.000000
                         GLP                 0.0        97    0.000000
                         GNC                 0.0       187    0.000000
                         GNL                 0.0         3    0.000000
                         HEV                 0.0       153    0.000000
              Eléctrico  BEV                 0.0    101212    0.000000
                         Hidrógeno           0.0         4    0.000000
                         PHEV                0.0        96    0.000000
                         REEV                0.0       112    0.000000
SÍ            Combustión Combustión  6470

In [41]:
# marquem el BEV amb 0 emissions com a emisions registrades:
df.loc[df["TECNOLOGÍA"]=="BEV", ["REG_EMISIONES"]] = "SÍ"

# guardem les mitjanes per a cada mes per a cada tipus de motor/tecnolog.
mitjanes = df.groupby(
        ["REG_EMISIONES", "PERIODO", "MOTOR", "COMBUSTIBLE", "TECNOLOGÍA",]
    )[["CO2_MEDIA"]].mean()
mitjanes = mitjanes.loc[("SÍ",),:]

# omplim els valros de la mitjana per als vehicles que no tenien emissions registrades
for mes, motor, combustible, tecnologia in mitjanes.index:
    df.loc[
        (df["PERIODO"]==mes) & 
        (df["MOTOR"]==motor) & 
        (df["COMBUSTIBLE"]==combustible) & 
        (df["TECNOLOGÍA"]==tecnologia) & 
        (df["REG_EMISIONES"]=="NO"), 
        "CO2_MEDIA"] = mitjanes.loc[(mes, motor, combustible, tecnologia), "CO2_MEDIA"]


In [45]:
# df2 = df.copy(deep=True)
# df = df2

# es desa el dataset amb dades mensuals per poblacions
df.to_csv(file_dataset, sep=',', index=False)


In [ ]:

df["CO2_total"] = df["CO2_MEDIA"] * df["CANTIDAD"]
df["CO2_mean"] = df["CO2"] / df["CANTIDAD"]
df_t = df.groupby(["REG_EMISIONES", "MOTOR", "TECNOLOGÍA"]).agg({
    "CANTIDAD": ["sum",],
    "CO2": ["sum", "mean"],
    "CO2_MEDIA": ["sum", "mean"],
    "CO2_total": ["sum"],
    "CO2_mean": ["sum", "mean"],
})

# df_t


In [ ]:
# Càlcul del total d'emission
df["CO2"] = df["CO2_MEDIA"] * df["CANTIDAD"]

# df
# df_t = df.groupby(["REG_EMISIONES", "MOTOR", "TECNOLOGÍA"])[["CO2","CANTIDAD"]].sum()
# df_t["CO2_MEDIA"] = df_t["CO2"] / df_t["CANTIDAD"]
# df_t


In [ ]:

# Comprovació de totals per motor/combustible/tecnologia
# df.groupby(['MOTOR', 'COMBUSTIBLE', 'TECNOLOGÍA',])[['CO2', 'CANTIDAD', '%']].sum().reset_index()


In [ ]:
# Agrupació per tecnologia

df_tecno = df.groupby(['PERIODO', 'TECNOLOGÍA',]
    )[['CANTIDAD']].sum().reset_index()
# print(df_tecno.to_string())


In [ ]:
# crear dataset per a dades per periodes i tipus de tecnologia

periodes = pd.unique(df_tecno['PERIODO'])
tecnologies = pd.unique(df_tecno['TECNOLOGÍA'])
# print(periodes, motors)
df_group_tecno = pd.DataFrame()
for tecnologia in tecnologies:
    df_tmp = df_tecno[df_tecno['TECNOLOGÍA']==tecnologia].set_index('PERIODO')[['CANTIDAD']]
    df_tmp = df_tmp.rename(columns={'CANTIDAD': tecnologia})
    df_group_tecno = pd.concat([df_group_tecno, df_tmp,], axis=1)

df_group_tecno = df_group_tecno.fillna(0)
df_group_tecno = df_group_tecno[['Combustión', 'GLP', 'GNC', 'GNL', 'HEV', 'Hidrógeno', 'REEV', 'PHEV', 'BEV', ]]

for field in df_group_tecno.columns:
    df_group_tecno[f"{field} (Total)"] = df_group_tecno[field].cumsum()

df_group_tecno.to_csv(file_group_tecno, sep=',') #, index=False)
# df_group_tecno


In [ ]:
# crear dataset per provincies
# agrupem les dades per provincies: núm de BEV, PHEV, total emisions, matriculacions totals, dades agregades

provincies = pd.unique(df['PROVINCIA'])
fields_group = ['PROVINCIA', 'PERIODO',]

df_p = df[df['PROVINCIA']!='[N/D]']
df_prov = df[df['PROVINCIA']!='[N/D]']

df_prov = df_prov.groupby(fields_group)[['CANTIDAD', 'CO2', ]].sum()
df_prov['CO2 MEDIA'] = df_prov['CO2'] / df_prov['CANTIDAD']

#############################################################################
#print(df_prov.reset_index().groupby(['PROVINCIA'])[['CANTIDAD']].sum())

df_prov = df_prov.rename(columns={'CANTIDAD': 'MATRICULACIONES'})

df_bev = df_p[df_p['TECNOLOGÍA']=='BEV'].groupby(fields_group)[['CANTIDAD']].sum().rename(columns={'CANTIDAD': 'BEV'})
df_phev = df_p[df_p['TECNOLOGÍA']=='PHEV'].groupby(fields_group)[['CANTIDAD']].sum().rename(columns={'CANTIDAD': 'PHEV'})

print(df_bev.reset_index().groupby('PROVINCIA').sum())

df_prov = pd.concat([df_prov, df_bev, df_phev], axis=1)

df_prov['EV'] = df_prov['BEV'] + df_prov['PHEV']
df_prov['EV PORCENTAJE'] = (df_prov['EV'] / df_prov['MATRICULACIONES'] * 100)
df_prov['BEV PORCENTAJE'] = (df_prov['BEV'] / df_prov['MATRICULACIONES'] * 100)
df_prov['CO2'] = (df_prov['CO2'] / 1000).round(2)

# for provincia in provincies:
for field in ['MATRICULACIONES', 'CO2', 'BEV', 'PHEV', 'EV', ]:
    df_prov[f"{field} (Total)"] = df_prov[field].cumsum()

df_prov['BEV PORCENTAJE (Total)'] = df_prov['BEV (Total)'] / df_prov['MATRICULACIONES (Total)']
df_prov['PHEV PORCENTAJE (Total)'] = df_prov['PHEV (Total)'] / df_prov['MATRICULACIONES (Total)']
df_prov['EV PORCENTAJE (Total)'] = df_prov['EV (Total)'] / df_prov['MATRICULACIONES (Total)']

df_prov = df_prov.fillna(0)
df_prov = df_prov.round(2)

for field in ['MATRICULACIONES','CO2 MEDIA', 'BEV', 'PHEV', 'EV', 'BEV (Total)', 'PHEV (Total)', 'EV (Total)', ]:
    df_prov[field] = df_prov[field].astype(int)

df_prov = df_prov.rename(columns={'CO2': 'TOTAL CO2 kg/km'})
df_prov = df_prov.rename(columns={'CO2 MEDIA': 'CO2 MEDIA g/km'})

df_prov = df_prov.reset_index()
df_prov['COMUNIDAD'] = df_prov['PROVINCIA'].map(dict_comunitat)
df_prov.to_csv(file_group_prov, sep=',', index=False)
print(df_prov.to_string())


C:\Users\JoseV\AppData\Local\Temp\ipykernel_19508\54355226.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print(df_bev.reset_index().groupby('PROVINCIA').sum())


                          BEV
PROVINCIA                    
Alacant                  3021
Albacete                  433
Almería                   494
Araba                     601
Asturias                 1284
Badajoz                   565
Balears (Illes)          3378
Barcelona               13958
Bizkaia                  1519
Burgos                    439
Cantabria                 753
Castelló                  864
Ceuta                      36
Ciudad Real               272
Coruña (A)               1041
Cuenca                    119
Cáceres                   194
Cádiz                     882
Córdoba                   551
Gipuzkoa                  752
Girona                   1466
Granada                   763
Guadalajara               365
Huelva                    271
Huesca                    237
Jaén                      294
León                      307
Lleida                    787
Lugo                      406
Madrid                  42164
Melilla                    35
Murcia    

In [ ]:
# crear dataset d'emisions per a dades per periodes 

df_co2 = df.groupby(['PERIODO', 'TECNOLOGÍA',]
    )[['CO2', 'CANTIDAD']].sum().reset_index()

df_group_co2 = pd.DataFrame()
for tecnologia in tecnologies:
    df_tmp = df_co2[df_co2['TECNOLOGÍA']==tecnologia].set_index('PERIODO')[['CO2', 'CANTIDAD']]
    df_tmp = df_tmp.rename(columns={'CO2': f'CO2 {tecnologia}', 'CANTIDAD': f'VEHICULOS {tecnologia}'})
    df_tmp[f"CO2 {tecnologia} mitjana"] = df_tmp[f'CO2 {tecnologia}'] / df_tmp[f'VEHICULOS {tecnologia}']
    df_group_co2 = pd.concat([df_group_co2, df_tmp,], axis=1)

df_group_co2 = df_group_co2.fillna(0)
# df_group_co2 = df_group_co2[['Combustión', 'GLP', 'GNC', 'GNL', 'HEV', 'Hidrógeno', 'REEV', 'PHEV', 'BEV', ]]

df_group_co2.to_csv(file_group_co2, sep=',') #, index=False)
# df_group_co2

In [ ]:

df = df.groupby(['FECHA', 'PERIODO', 'PROVINCIA', 'POBLACIÓN', 
                 'MOTOR', 'COMBUSTIBLE', 'TECNOLOGÍA',]
    )[['CO2', 'CANTIDAD', '%']].sum().reset_index()
df

In [ ]:
# agrupar dades mensuals
df_group_mensual = df.groupby(['FECHA', 'PERIODO'] + fields_group)[fields_num].sum().reset_index()
# df_group_mensual.info()
df_group_mensual.to_csv(file_group_mensual, sep=",", index=False)

# agrupar dades totals
df_group_total = df.groupby(fields_group)[fields_num].sum().reset_index()
df_group_total["CO2"] = df_group_total["CO2"] / df_group_total["CANTIDAD"]

total_sum = df_group_total["CANTIDAD"].sum()
df_group_total["%"] = df_group_total["CANTIDAD"] / total_sum * 100
df_group_total = df_group_total[df_group_total["%"]>0.1]
df_group_total["%"] = df_group_total["%"].map('{:,.2f}'.format)

df_group_total.to_csv(file_group_total, sep=",", index=False)

df_group_total.info


In [ ]:
df